In [392]:
import sqlite3
import csv, re

# Open csv file
with open("energy_storage_db.csv", 'rt') as f:
    data = list(csv.reader(f))

# Format for storage in SQLite3 db
for i in range(0, len(data[0])):
    data[0][i] = re.sub(r"\s+", '_', data[0][i])

# Open connection to db and create a pointer to the cursor
conn = sqlite3.connect('energy_storage3.db')
c = conn.cursor()

# Delete the Table if it exists
c.execute('''DROP TABLE IF EXISTS projects''')

# Create new table with the following headers
c.execute('''CREATE TABLE projects (
   id integer primary key
  ,Project_Name text
  ,Technology_Type text
  ,Technology_Type_Category_1 text
  ,Technology_Type_Category_2 text
  ,Rated_Power_in_kW real
  ,Duration real
  ,Status text
  ,ServiceUse_Case_1 text
  ,ServiceUse_Case_2 text
  ,ServiceUse_Case_3 text
  ,ServiceUse_Case_4 text
  ,ServiceUse_Case_5 text
  ,ServiceUse_Case_6 text
  ,ServiceUse_Case_7 text
  ,ServiceUse_Case_8 text
  ,ServiceUse_Case_9 text
  ,ServiceUse_Case_10 text
  ,ServiceUse_Case_11 text
  ,ServiceUse_Case_12 text
  ,Latitude real
  ,Longitude real
  ,City text
  ,StateProvince text
  ,Country text
  ,Street_Address text
  ,ZipMail_Code text
  ,Description text
  ,Web_Link_1 text
  ,Announcement_Date text
  ,Construction_Date text
  ,Commissioning_Date text
  ,Decommissioning_Date text
  ,ISORTO text
  ,Utility text
  ,Utility_Type text
  ,Grid_Interconnection text
  ,Paired_Grid_Resource text
  ,Ownership_Model text
  ,Equity_Owner_1 text
  ,Equity_Owner_1_Percentage real
  ,Equity_Owner_2 text
  ,Equity_Owner_2_Percentage real
  ,Energy_Storage_Technology_Provider text
  ,Power_Electronics_Provider text
  ,Integrator_Company text
  ,OM_Contractor text
  ,Developer text
  ,EPC_1 text
  ,EPC_2 text
  ,EPC_3 text
  ,Debt_Provider text
  ,Projected_Project_Lifetime_years real
  ,Performance text
  ,Capital_Expenditure real
  ,Operating_Expense_kWh_CapacityYear real
  ,Funding_Source_1 text
  ,Funding_Source_Details_1 text
  ,Funding_Amount_1 real
  ,Funding_Source_2 text
  ,Funding_Source_Details_2 text
  ,Funding_Amount_2 real
  ,Funding_Source_3 text
  ,Funding_Source_Details_3 text
  ,Funding_Amount_3 real
  ,Research_Institution text
  ,Research_Description text
  ,Research_Institution_Link text
  ,Contact_Name text
  ,Contact_Email text
  ,Contact_Phone text
  ,Contact_Street_Address text
  ,Contact_City text
  ,Contact_StateProvince text
  ,Contact_ZipMail_Code text
  ,Contact_Country text
  ,Record_Created text
  ,Last_Updated text
  ,Black_Start text
  ,Electric_Supply_Reserve_Capacity_NonSpinning text
  ,Electric_Supply_Reserve_Capacity_Spinning text
  ,Load_Following_Tertiary_Balancing text
  ,Ramping text
  ,Voltage_Support text
  ,Electric_Energy_Time_Shift text
  ,Electric_Supply_Capacity text
  ,Transmission_Congestion_Relief text
  ,Transmission_Support text
  ,Renewables_Capacity_Firming text
  ,Distribution_upgrade_due_to_solar text
  ,Distribution_upgrade_due_to_wind text
  ,Transmission_upgrades_due_to_solar text
  ,Transmission_upgrades_due_to_wind text
  ,Electric_Bill_Management text
  ,GridConnected_Commercial_Reliability_Quality text
  ,GridConnected_Residential_Reliability text
  ,Frequency_Regulation text
  ,Transportable_TransmissionDistribution_Upgrade_Deferral text
  ,Stationary_TransmissionDistribution_Upgrade_Deferral text
  ,Onsite_Renewable_Generation_Shifting text
  ,Electric_Bill_Management_with_Renewables text
  ,Renewables_Energy_Time_Shift text
  ,OnSite_Power text
  ,Transportation_Services text
  ,Microgrid_Capability text
  ,Resiliency text
  ,Demand_Response text
)''')

# Create an index column and unique id for each entry in the data table
for i in range(0,len(data)):
    if i != 0:
        for jj in range(len(data[0]),0,-1):
            j = jj - 1
            if jj == len(data[0]):
                data[i].append(data[i][j])
            else:
                data[i][jj] = data[i][j]
        data[i][0] = i

# Enter the formatted data table into the db
c.executemany('INSERT INTO projects VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', data[1:len(data)])

# Test to check data uploaded correctly (query can be anything)
'''
for i in range(1,len(data)):
    c.execute('SELECT id, Rated_Power_in_kW FROM projects WHERE id=?', (i,))
    raw = c.fetchall()
    print(raw[0][1])
'''

conn.commit()
conn.close()

In [394]:
import plotly.tools as tls
tls.embed('https://plot.ly/~cufflinks/8')

import plotly.plotly as py
! pip install cufflinks --upgrade

import cufflinks as cf
import pandas as pd
import numpy as np

import sqlite3
conn = sqlite3.connect('energy_storage3.db')
c = conn.cursor()

def delete_nulls ( df, col_name ):
    "delete rows with null values in column 'col_name' of date frame 'df'"
    indices = [0]
    for i in range(0,len(df)):
        ii = i+1
        series = df[col_name]
        if series[i] == '':
            indices.extend([i])

    indices = indices[1:len(indices)]
    modified_df = df[0:0]
    last = 0
    for i in indices:
        if i == 0:
            last = -1
        modified_df = pd.concat([modified_df, df[(last+1):i]])
        last = i
    return pd.concat([modified_df, df[last+1:len(df)]])

# Query db for data of interest
master_df = pd.read_sql_query("SELECT Technology_Type, Rated_Power_in_kW, Capital_Expenditure FROM projects ORDER BY Technology_Type", conn, coerce_float=True)

# Clean up data
master_df = delete_nulls(master_df, 'Rated_Power_in_kW')

# Group installations by Technology Type and calculate total capacity of type and count
grouped = master_df.groupby('Technology_Type')
types_df = pd.DataFrame({"Total_Power": grouped.Rated_Power_in_kW.sum(),
                       "Number_of_Installations": grouped.Technology_Type.count()})

# Calculate avg storage of installation by type
types_df = types_df.sort_values('Number_of_Installations', ascending=False)
avg_power_s = new_df.apply(lambda x: x[1] / x[0], axis=1)


# Chart average storage of installation by Technoloty Type
avg_power_s.iplot(kind='bar', yTitle='Power [kW]', title='US Enery Storage Capacity by Type',
             filename='Avg-Cap-By-Type-bar-chart')

# Chart number of installations by type with Plotly
types_df['Number_of_Installations'].iplot(kind='bar', yTitle='Number of Installations', title='US Enery Storage Installations by Type',
             filename='Count-By-Type-bar-chart')


Requirement already up-to-date: cufflinks in /Users/joshuamarcley/developer/anaconda3/lib/python3.5/site-packages
Requirement already up-to-date: plotly>=1.7.6 in /Users/joshuamarcley/developer/anaconda3/lib/python3.5/site-packages (from cufflinks)
Requirement already up-to-date: colorlover>=0.2 in /Users/joshuamarcley/developer/anaconda3/lib/python3.5/site-packages (from cufflinks)
Requirement already up-to-date: pandas in /Users/joshuamarcley/developer/anaconda3/lib/python3.5/site-packages (from cufflinks)
Requirement already up-to-date: six in /Users/joshuamarcley/developer/anaconda3/lib/python3.5/site-packages (from plotly>=1.7.6->cufflinks)
Requirement already up-to-date: pytz in /Users/joshuamarcley/developer/anaconda3/lib/python3.5/site-packages (from plotly>=1.7.6->cufflinks)
Requirement already up-to-date: requests in /Users/joshuamarcley/developer/anaconda3/lib/python3.5/site-packages (from plotly>=1.7.6->cufflinks)
Requirement already up-to-date: numpy>=1.7.0 in /Users/joshu

In [253]:
import plotly.tools as tls
tls.set_credentials_file(username='JMarcley', api_key='ey561ff4zv')

In [245]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__)

init_notebook_mode(connected=True)

1.12.9
